In [5]:
import os
import leafmap
from samgeo import SamGeo, tms_to_geotiff
from datetime import datetime
import json

In [6]:
# Obtendo a data e hora atual
now = datetime.now()
formatted_date = now.strftime("%Y_%m_%d_%H_%M")

def extract_coordinates(geojson_file):
    with open(geojson_file, 'r') as file:
        data = json.load(file)

    point_coords = []
    for feature in data['features']:
        if feature['geometry']['type'] == 'Point':
            point_coords.append(feature['geometry']['coordinates'])

    return point_coords

### Modelo

In [7]:
# vit_b(91M), vit_l(308M), vit_h(636M)

sam = SamGeo(
    model_type="vit_b",
    automatic=False,
    sam_kwargs=None,
)

### Imagem a ser segmentada

In [8]:
# imagem

image = 'img_teste.tif'

# pasta saída, arquivo saída

pasta_saida = f'saida/mascaras_por_ponto/{os.path.splitext(os.path.basename(image))[0]}'

os.makedirs(pasta_saida, exist_ok=True)

mascaras_geradas = f'{pasta_saida}/mascaras_{formatted_date}.tif'

In [9]:
sam.set_image(image)

In [10]:
# pontos e detecção

point_coords = extract_coordinates('pontos_amostragem.geojson')

print(point_coords)

[[-48.6293, -26.957], [-48.6293, -26.9572], [-48.6292, -26.9575]]


In [11]:
sam.predict(point_coords, point_labels=1, point_crs="EPSG:4326", output=mascaras_geradas)


In [12]:
m = leafmap.Map(center=point_coords[0], zoom=19, height="400px")
m.add_basemap("SATELLITE")
m.layers[-1].visible = False

m.add_raster(image, layer_name="Image")

m.add_raster(mascaras_geradas, layer_name="Mask1", nodata=0, cmap="Blues", opacity=1)

m

Map(center=[-48.6293, -26.957], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…